In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import plotly.offline as py
import plotly.express as px

In [3]:
Image_Width = 64
Image_Height = 64
Cannels = 3

INPUT_SHAPE = (Image_Width, Image_Height, Cannels)
NB_CLASSES = 2
EPOCHS = 15
BATCH_SIZE = 32

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(Conv2D(250,(3,3)))
model.add(Activation("relu"))
  
model.add(Conv2D(128,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))

model.add(Conv2D(256,(2,2)))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
    
model.add(Flatten())
model.add(Dense(32))
model.add(Dropout(0.25))
model.add(Dense(7))
model.add(Activation("softmax"))

2021-12-27 18:27:14.475049: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 250)       144250    
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 250)       0

In [7]:
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'adv_data/adv_train',
    target_size=(Image_Height, Image_Width),
    batch_size=BATCH_SIZE,
    class_mode='categorical')


Found 10000 images belonging to 7 classes.


In [8]:
validation_datagen = ImageDataGenerator(rescale = 1/255.0)


validation_generator = train_datagen.flow_from_directory(
    'adv_data/adv_val', 
    target_size=(Image_Height, Image_Width),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 1000 images belonging to 7 classes.


In [10]:
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics =['accuracy'])

In [11]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS)

2021-12-27 18:30:57.868248: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
312/312 [==============================] - 439s 1s/step - loss: 0.9267 - accuracy: 0.7022 - val_loss: 1.0239 - val_accuracy: 0.7067
Epoch 2/15
312/312 [==============================] - 410s 1s/step - loss: 0.8279 - accuracy: 0.7046 - val_loss: 1.0615 - val_accuracy: 0.6240
Epoch 3/15
312/312 [==============================] - 422s 1s/step - loss: 0.7935 - accuracy: 0.7084 - val_loss: 1.0634 - val_accuracy: 0.6966
Epoch 4/15
312/312 [==============================] - 446s 1s/step - loss: 0.7698 - accuracy: 0.7062 - val_loss: 1.1346 - val_accuracy: 0.6532
Epoch 5/15
312/312 [==============================] - 360s 1s/step - loss: 0.7511 - accuracy: 0.7110 - val_loss: 1.0987 - val_accuracy: 0.6835
Epoch 6/15
312/312 [==============================] - 294s 942ms/step - loss: 0.7343 - accuracy: 0.7162 - val_loss: 1.3380 - val_accuracy: 0.6623
Epoch 7/15
312/312 [==============================] - 302s 966ms/step - loss: 0.7188 - accuracy: 0.7188 - val_loss: 1.1254 - val_accuracy: 

In [14]:
## Test Performance on Test Data
from keras.preprocessing.image import load_img, img_to_array

def preprocess_image(path):
    img = load_img(path, target_size = (Image_Height, Image_Width))
    a = img_to_array(img)
    a = np.expand_dims(a, axis = 0)
    a /= 255.
    return a

In [24]:
test_images = ['HW4_data/private/test_{i}.jpg'.format(i=i) for i in range(1500)]
test_preprocessed_images = np.vstack([preprocess_image(fn) for fn in test_images])

In [23]:
print(validation_generator.class_indices)


{'Atelectasis': 0, 'Cardiomegaly': 1, 'Edema': 2, 'Lung Opacity': 3, 'No Finding': 4, 'Pleural Effusion': 5, 'Support Devices': 6}


In [25]:
array = model.predict(test_preprocessed_images, batch_size=1, verbose=1)
answer = np.argmax(array, axis=1)
print(len(answer))

1500/1500 [==============================] - 15s 10ms/step
1500


In [29]:
final = []

for i in range(1500):
    label = answer[i]
    if label == 0:
        arr = [1, 0, 0, 0, 0, 0, 0]
    elif label == 1:
        arr = [0, 1, 0, 0, 0, 0, 0]
    elif label == 2:
        arr = [0, 0, 1, 0, 0, 0, 0]    
    elif label == 3:
        arr = [0, 0, 0, 1, 0, 0, 0] 
    elif label == 4:
        arr = [0, 0, 0, 0, 1, 0, 0]
    elif label == 5:
        arr = [0, 0, 0, 0, 0, 1, 0]
    else:
        arr = [0, 0, 0, 0, 0, 0, 1]

    final.append(arr)  

In [30]:
data = np.array(final)
df = pd.DataFrame(data, columns = ['Atelectasis', 'Cardiomegaly','Edema','Lung Opacity','No Finding','Pleural Effusion','Support Devices'])
print(df)

      Atelectasis  Cardiomegaly  Edema  Lung Opacity  No Finding  \
0               0             0      0             0           1   
1               0             0      0             0           1   
2               0             0      0             0           1   
3               0             0      0             0           1   
4               0             0      0             0           1   
...           ...           ...    ...           ...         ...   
1495            0             0      1             0           0   
1496            0             0      0             0           1   
1497            0             0      0             0           1   
1498            0             0      0             0           1   
1499            0             0      0             0           1   

      Pleural Effusion  Support Devices  
0                    0                0  
1                    0                0  
2                    0                0  
3              

In [ ]:
df.to_csv('[106091228]_advanced_prediction.csv')